In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import wandb

from config import config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Loading, scaling the data
data has been explored and concaternated in `data_preparerationipynb`

In [ ]:
# Load the dataset
df = pd.read_csv('../data/train.csv')

In [ ]:
# Separate the target variable from the features
X = df.drop('price_range', axis=1)

enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(df['price_range'].values.reshape(-1, 1))

#  Split the data into train, validation, and test sets 0.7, 0.15, 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=config['seed'])
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=config['seed'])

# Create scaling pipeline for the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Sweep initialization
sweep_id = wandb.sweep(sweep=config, project="phone-price-prediction")
name = 'TensorFlow Hidden={}'.format(str(config['hidden_layers']))

def main():
    wandb.init(project='phone-price-prediction', name=name, config=config)

    # Create the data loaders
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train), seed=config['seed']).batch(wandb.config.batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(wandb.config.batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(wandb.config.batch_size)

    # Build the model
    input_shape = (20,)
    input = tf.keras.layers.Dense(20, input_shape=input_shape, activation='relu'),
    model = tf.keras.Sequential(input)

    for size in config['hidden_layers']:
        model.add(tf.keras.layers.Dense(size, activation='relu'))
        model.add(tf.keras.layers.Dropout(wandb.config.dropout))

    output = tf.keras.layers.Dense(4, activation='softmax')
    model.add(output)

    # Create optimizer and compile the model
    optimizer = tf.keras.optimizers.Adam(beta_1=config['beta1'], beta_2=config['beta2'], epsilon=config['epsilon'], learning_rate=wandb.config.lr)
    model.compile(optimizer=optimizer, loss=config['loss_keras'], metrics=['accuracy'])

    # Create a callback
    wandb_callback = wandb.keras.WandbCallback()

    # Train the model
    model.fit(train_dataset, epochs=wandb.config.epochs, validation_data=val_dataset, callbacks=[wandb_callback])

    # build, compile, and train model here
    test_loss, test_acc = model.evaluate(test_dataset)

    # Make a log into the wandb
    wandb.log({'test_loss': test_loss, 'test_acc': test_acc})
    wandb.finish()
wandb.agent(sweep_id, function=main, count = 4)